In [1]:
# Store technologies in Chroma vector DB using Gemini embeddings via LangChain
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from technologies import technologies\

all_docs = []
for category in ["frontend", "backend", "tools"]:
    for tech in technologies[category]:
        all_docs.append(
            Document(
                page_content=tech,
                metadata={"category": category}
            )
        )
print(all_docs)


[Document(metadata={'category': 'frontend'}, page_content='React'), Document(metadata={'category': 'frontend'}, page_content='Angular'), Document(metadata={'category': 'frontend'}, page_content='Vue.js'), Document(metadata={'category': 'frontend'}, page_content='Svelte'), Document(metadata={'category': 'frontend'}, page_content='Preact'), Document(metadata={'category': 'frontend'}, page_content='Ember.js'), Document(metadata={'category': 'frontend'}, page_content='Backbone.js'), Document(metadata={'category': 'frontend'}, page_content='jQuery'), Document(metadata={'category': 'frontend'}, page_content='SolidJS'), Document(metadata={'category': 'frontend'}, page_content='Qwik'), Document(metadata={'category': 'frontend'}, page_content='Alpine.js'), Document(metadata={'category': 'frontend'}, page_content='Bootstrap'), Document(metadata={'category': 'frontend'}, page_content='Tailwind CSS'), Document(metadata={'category': 'frontend'}, page_content='Materialize'), Document(metadata={'cate

In [2]:

vectorstore = Chroma.from_documents(
    all_docs,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory="tech_dump_vector_store"
)
print("Single Chroma collection created with", len(all_docs), "technologies across frontend, backend, and tools.")

e:\OpenSourceHunt\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Single Chroma collection created with 181 technologies across frontend, backend, and tools.


In [3]:
res=vectorstore.similarity_search_with_score(
    query="Javascript",
    k=5
)
print(res)


[(Document(metadata={'category': 'frontend'}, page_content='JavaScript (ES6+)'), 0.5591710805892944), (Document(metadata={'category': 'frontend'}, page_content='jQuery'), 0.6543084383010864), (Document(metadata={'category': 'backend'}, page_content='Node.js'), 0.7669662833213806), (Document(metadata={'category': 'frontend'}, page_content='HTML5'), 0.9442819356918335), (Document(metadata={'category': 'backend'}, page_content='Express.js'), 0.9840607643127441)]
